# 2024-2 URP assignment

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

In [2]:
# 데이터프레임 선언
Q_df = pd.DataFrame(columns = ['문서 번호', '제목',  '질문',  "위치 고유번호" , '조회수', '질문 날짜'])
A_df = pd.DataFrame(columns = ['문서 번호', '답변 ID',  '답변', '답변 날짜'])
loc_dict = pd.read_csv("naver_kin_regions_df_l2.csv", index_col = "지역 코드")

In [3]:
local_lists = loc_dict.index.tolist()
local_lists

[120115,
 120116,
 120117,
 120118,
 120119,
 120120,
 120121,
 120122,
 120123,
 120124,
 120125,
 120126,
 120127,
 120128,
 120129,
 120130,
 120131,
 120132,
 120133,
 120134,
 120135,
 120136,
 120137,
 120138,
 120139,
 120203,
 120204,
 120205,
 120206,
 120207,
 120208,
 120209,
 120210,
 120211,
 120212,
 120213,
 120214,
 120215,
 120216,
 120217,
 120218,
 120306,
 120307,
 120308,
 120309,
 120310,
 120311,
 120312,
 120313,
 120404,
 120405,
 120406,
 120407,
 120408,
 120409,
 120410,
 120411,
 120412,
 120413,
 120504,
 120505,
 120506,
 120507,
 120508,
 120603,
 120604,
 120605,
 120606,
 120607,
 120701,
 120702,
 120703,
 120704,
 120705,
 120802,
 120803,
 120804,
 120805,
 120806,
 120807,
 120808,
 120809,
 120810,
 120811,
 120812,
 120813,
 120814,
 120815,
 120816,
 120817,
 120818,
 120819,
 120905,
 120906,
 120907,
 120908,
 120909,
 120910,
 120911,
 120912,
 120913,
 120914,
 120915,
 120916,
 120917,
 120918,
 120919,
 120920,
 120921,
 120922,
 120923,
 

# URL에서 dirID 값을 가져오는 함수

In [4]:
def get_dirid(url):
    match = re.search(r'dirId=(\d+)', url)
    dirid_value = match.group(1)
    if len(dirid_value) >= 6 :
        dirid_value = dirid_value[:6]
    return dirid_value

# URL에서 docID 값을 가져오는 함수

In [5]:
def get_docid(url):
    match = re.search(r'docId=(\d+)', url)
    docid_value = match.group(1)
    return docid_value

# 반복되는 패턴을 통해서 docID를 통해 위치 반환하는 함수(추후 매핑 테이블을 이용할 예정)

In [6]:
# dirID -> loc
def get_loc(id=""):
    url = 'https://kin.naver.com/qna/list.naver?dirId=12'+id

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    location = soup.find(attrs={'class': 'location'}).get_text(strip=True)
    return location

In [7]:
def find_pattern(text):
    # 문자열 길이의 절반까지 탐색(더 긴 패턴은 두 번 반복될 수 없으므로)
    for length in range(1, len(text)):
        # 길이 `length`만큼 패턴을 잘라서 비교
        pattern = text[:length]
        # 패턴이 두 번 연속 반복되는지 확인
        if text.startswith(pattern * 2):
            return pattern

In [8]:
# 지역 & 플레이스 기본 정보 문자열 제거를 위한 변수 지정
url2 = 'https://kin.naver.com/qna/list.naver?dirId=12'
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response2 = requests.get(url2, headers=headers2)
soup2 = BeautifulSoup(response2.text, 'html.parser')
useless_information = soup2.find(attrs={'class': 'location'}).get_text(strip=True)

def return_location(search):
    a = get_loc(search[:-2]+"01").replace(get_loc(search[:-2]),"")
    a_elim = get_loc(search[:-len(search)]+search[-len(search):-2]).replace(get_loc(search[:-len(search)]),"")
    first_loc = find_pattern(a)
    a = a[len(first_loc):]
    b = get_loc(search).replace(a,"")
    c = b.replace(useless_information, "")
    return c.replace(a_elim,"")

def return_full_location(search):
    full_location = ""
    n = 2
    while n <= len(search):
        full_location += return_location(search[:n])+ " "
        n += 2
    return full_location

# 최종적으로 질문과 답변 정보 받아오는 함수

In [9]:
def collect_information(url, Q_df, A_df):
    # 페이지 요청
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    # BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 질문 제목 추출
    title = soup.find(attrs={'class': 'endTitleSection'}).get_text(strip=True)
    title = title.replace(",","")
    Q_detail= soup.find(attrs={'class': 'questionDetail'}).get_text(strip=True,separator=' ')
    Q_detail = Q_detail.replace(",","")
    Q_info = soup.find_all(class_='infoItem')
    Q_view = Q_info[0].get_text()[3:]
    Q_date = Q_info[1].get_text()[3:]
    if Q_date.endswith(" 전"):
        Q_date = datetime.today().strftime('%Y.%m.%d')
    if Q_date.startswith("작성일"):
        Q_date = Q_date.replace("작성일", "")
    ans = []
    A_dates = []
    try:
        answers = soup.find_all(class_='se-main-container')
        ans_date = soup.find_all(class_='answerDate')
        for answer in answers:
            ans.append(answer.get_text(strip=True).replace('\u200b', ''))
        for ans_dates in ans_date:
            A_dates.append(ans_dates.get_text(strip=True))
    except:
        answer_error = "아직 답변이 없습니다"
    
    ID = get_dirid(url)
    doc_ID = get_docid(url)
    Q_inf_list =[get_docid(url), title, Q_detail, ID, Q_view, Q_date]
    A_inf_list =[]
    n = 1
    for i,j in zip(ans,A_dates):
        changed = i.replace(","," ")
        if j.endswith(" 전"):
            today = datetime.today().strftime('%Y.%m.%d.')
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],today])
        else:
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],j])
        n += 1
    Q_df.loc[Q_df.shape[0]] = Q_inf_list
    
    for i in A_inf_list:
        A_df.loc[A_df.shape[0]] = i
    return Q_df, A_df

# 질문 리스트에서 각 질문의 URL 값 가져오기

In [10]:
# 질문 목록에서 URL만 가져기기
urls = []
def get_url_from_list(search_url):
    global urls
    response = requests.get(search_url) 
    
    soup = BeautifulSoup(response.text, 'html.parser')

    elements = soup.find_all(class_=re.compile(r'^_nclicks:kls_new\.list'))
    
    for element in elements[1:]:
        href = "https://kin.naver.com"+element.get('href')
        urls.append(href)
    return urls

### list 페이지 설정
### 삭제 직전인 데이터는 못불러옴 예시로 보여주는 수 밖에 없다. 글로 설명 불가능

In [11]:
def collect_urls(region):
    try:
        for i in range(1,21):
            page = f"https://kin.naver.com/qna/list.naver?dirId={region}&queryTime=2024-11-06%2017%3A33%3A26&page={i}"
            urls= get_url_from_list(page)
    except:
        pass
    return urls

# 11월 06일 기준 전체 지역 URL 크롤링(매핑테이블의 index를 사용) -> 현재 지역&플레이스에 나와있는 모든 질문 url을 수집
## 총 실행시간 약 1시간30분

In [12]:
print(local_lists)

[120115, 120116, 120117, 120118, 120119, 120120, 120121, 120122, 120123, 120124, 120125, 120126, 120127, 120128, 120129, 120130, 120131, 120132, 120133, 120134, 120135, 120136, 120137, 120138, 120139, 120203, 120204, 120205, 120206, 120207, 120208, 120209, 120210, 120211, 120212, 120213, 120214, 120215, 120216, 120217, 120218, 120306, 120307, 120308, 120309, 120310, 120311, 120312, 120313, 120404, 120405, 120406, 120407, 120408, 120409, 120410, 120411, 120412, 120413, 120504, 120505, 120506, 120507, 120508, 120603, 120604, 120605, 120606, 120607, 120701, 120702, 120703, 120704, 120705, 120802, 120803, 120804, 120805, 120806, 120807, 120808, 120809, 120810, 120811, 120812, 120813, 120814, 120815, 120816, 120817, 120818, 120819, 120905, 120906, 120907, 120908, 120909, 120910, 120911, 120912, 120913, 120914, 120915, 120916, 120917, 120918, 120919, 120920, 120921, 120922, 120923, 120924, 120925, 120926, 120927, 120928, 120929, 120930, 120931, 120932, 120933, 120934, 120935, 120936, 121001,

In [13]:
for region in tqdm(local_lists, desc="Processing URLs", ncols=100, leave=True):
    collect_urls(region)

Processing URLs: 100%|██████████████████████████████████████████| 257/257 [1:22:35<00:00, 19.28s/it]


In [14]:
len(urls)

12290

In [15]:
urls = [url for url in urls if not isinstance(url, list)]
len(urls)

12290

# 실제 데이터 불러오기(매핑 테이블 추후 SQL에서 join 예정)
## 45분 정도 소요

In [16]:
for i in tqdm(range(len(urls)), desc="Processing URLs", ncols=100, leave=True):
    url = urls[i]
    collect_information(url, Q_df, A_df)

Processing URLs: 100%|████████████████████████████████████████| 12290/12290 [44:09<00:00,  4.64it/s]


In [17]:
Q_df

,문서 번호,제목,질문,위치 고유번호,조회수,질문 날짜
0,477577274,질문강남 고터 지하상가,고터 지하상가 11월 14일 목요일 수능일때 휴무인가요?,120115,15,2024.11.07
1,477576108,질문압구정로데오역 물품보관함,현제 압구정로데오역에 물품보관함 있나요? 있으면 어느 위치에 있는건가요?,120115,15,2024.11.07
2,477573260,질문강남 110명가능한 회식장소 빔 프로젝터있는곳 추천 좀,안녕하세요. 12월에 회사 송년회를 할 예정인데 110명 정도 인원이 모일 장소를 ...,120115,13,2024.11.07
3,477570620,질문개포중학교 질문,개포중에 잘생긴사람 많나요..? 많진 않아도 있긴 한가요..?ㅠ,120115,19,2024.11.07
4,477561630,질문가격문의요,제주에서 강남까지 박스한개 가격과 소요시간문의요 접수는 언제까지 하는지도요,120115,14,2024.11.07
...,...,...,...,...,...,...
12285,476391816,질문번개장터 사기학생 학교 아는데 학교에 전화해도,제목 그대로 번개장터에서 사기를 당했는데 사기학생이 11년생 어디학교 몇반인지 알게...,121719,27,2024.10.09
12286,476537754,질문세종 CGV는 무인 주차장 인가요?,,121721,27,2024.10.14
12287,476260849,질문세종 몰리브 상가에 입점된 옷가게 브랜드는 뭐 뭐 있나...,세종 몰리브 상가에 입점된 옷가게 브랜드는 뭐 뭐 있나요,121721,30,2024.10.07
12288,475642460,질문종촌동 노래방,세종시 종촌동에 꿈끼카드 사용 가능한 노래방 있나요?,121721,17,2024.09.21


In [18]:
A_df

,문서 번호,답변 ID,답변,답변 날짜
0,477577274,4775772741,. 상관없습니다.,2024.11.07.
1,477577274,4775772742,. 상관없습니다.,2024.11.07.
2,477576108,4775761081,정로데오역에 물품보관함이 있습니다.역 내부에 위치해 있으니 참고하세요.,2024.11.07.
3,477561630,4775616301,안녕하세요?제주의 동명에 따라 달라집니다만 대략요금은 10만원입니다 접수는 언제든...,2024.11.07.
4,477558325,4775583251,군 강남구 서초구5학군 노원구 중계동,2024.11.07.
...,...,...,...,...
11956,475169557,4751695572,로 입주민분 자녀 소선여중 배정받았습니다.효목2동은 신천동과 달리 수성구학군 배정및...,2024.10.29.
11957,476649506,4766495061,1병 까지 콜키지 가능합니다2인 코스 2명이 드시기에 충분합니다,2024.10.15.
11958,476391816,4763918161,에 신고를 해서 경찰이 학교랑 연락해서 찾는게 맞아요학교 입장에선 수사기관도 아니고...,2024.10.12.
11959,476537754,4765377541,CGV 무인 주차장 있습니다,2024.10.14.


In [19]:
Q_df.to_csv('Questions.csv', index = False)
A_df.to_csv('Answers.csv', index = False)